Exercise: ConfigMap
---------------------------

The use of ConfigMap is to be demonstrated using the Apache Web Server as an example.

First the ConfigMaps have to be created using `kubectl`. Before that, let's create a new namespace `configmap` to separate build resources from the others:

In [ ]:
! kubectl create namespace configmap
! kubectl create configmap web2 --from-file=index=09-8-ConfigMap/index-2.html -n configmap
! kubectl create configmap web1 --from-file=index=09-8-ConfigMap/index-1.html -n configmap


We could also have described the ConfigMaps created in this way in YAML format. Therefore we output them in YAML format:

In [ ]:
! kubectl get configmap/web1 -o yaml -n configmap
! kubectl get configmap/web2 -o yaml -n configmap

The volumes in the YAML file must then be set in such a way that they access the ConfigMap keys

- - -

    apiVersion: v1
    kind: Pod
    metadata:
      labels:
        app.kubernetes.io/name: web1
      name: web1
      namespace: configmap  
    spec:
      containers:
      - image: httpd
        name: apache
        volumeMounts:    
         - name: config-volume
           mountPath: /usr/local/apache2/htdocs
      volumes:
        - name: config-volume
          configMap:
            name: web1
            items:
            - key: index
              path: index.html

- - -

Then we can create the pods and services:

In [ ]:
! kubectl apply -f 09-8-ConfigMap -n configmap
! kubectl get services --namespace configmap

We look at the output of the web server with the respective index.html files or ConfigMap

In [ ]:
! echo "Web Server 1: http://$(cat ~/work/server-ip):$(kubectl get service web1 -n configmap -o=jsonpath='{ .spec.ports[0].nodePort }')"
! echo "Web Server 2: http://$(cat ~/work/server-ip):$(kubectl get service web2 -n configmap -o=jsonpath='{ .spec.ports[0].nodePort }')"

And we look at the directory and file contents as the pods see them:

In [ ]:
! kubectl exec -n configmap web1 -- ls -la /usr/local/apache2/htdocs

In [ ]:
! kubectl exec -n configmap web1 -- cat /usr/local/apache2/htdocs/index.html

- - -
Clean up

In [ ]:
! kubectl delete namespace configmap